In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_cooccured/lyric_cooccured_by_line.csv"
words = pd.read_csv(url, encoding="utf-8").iloc[:, 1:]
words


,genre,year,song_id,title,line,noun,predicate
0,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),끝내(P)
1,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),돌아가(P)
2,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),끝내(P)
3,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),돌아가(P)
4,성인가요,1964.0,4083218.0,내일또 만납시다,2.0,하늘(N),반짝이(P)
...,...,...,...,...,...,...,...
182946,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,그늘(N),받(P)
182947,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),가리(P)
182948,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),받(P)
182949,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,미안(N),가리(P)


## 1. 5년 단위로 데이터 분리


In [2]:
word_fragments_by_year = [
    words.loc[words["year"].isin(period)]
    for period in [
        list(range(1995, 2000)),
        list(range(2000, 2005)),
        list(range(2005, 2010)),
        list(range(2010, 2015)),
        list(range(2015, 2020)),
    ]
]


## 2. 중복된 노드 가진 row 추출


In [3]:
duplicate_nodes_cache = {}
words_1995_2019 = words.loc[words["year"].isin(list(range(1995, 2020)))]
for _, word in words_1995_2019.iterrows():
    duplicate_nodes_cache.update(
        {
            (word["noun"], word["predicate"]): duplicate_nodes_cache.get(
                (word["noun"], word["predicate"]), 0
            )
            + 1
        }
    )
duplicate_nodes_cache = {
    key: val for key, val in duplicate_nodes_cache.items() if val > 1
}


In [4]:
common_words = words_1995_2019.loc[
    list(
        map(
            lambda x: x in duplicate_nodes_cache.keys(),
            zip(words_1995_2019["noun"], words_1995_2019["predicate"]),
        )
    )
]
common_words


,genre,year,song_id,title,line,noun,predicate
54127,댄스,1995.0,64313.0,Come Back Home,1.0,애(N),쓰(P)
54128,댄스,1995.0,64313.0,Come Back Home,5.0,가슴속(N),있(P)
54133,댄스,1995.0,64313.0,Come Back Home,8.0,날(N),기나길(P)
54134,댄스,1995.0,64313.0,Come Back Home,8.0,속(N),기나길(P)
54136,댄스,1995.0,64313.0,Come Back Home,9.0,자신(N),보(P)
...,...,...,...,...,...,...,...
179250,발라드,2019.0,31453551.0,흔한 이별,36.0,말(N),괜찮(P)
179251,발라드,2019.0,31453551.0,흔한 이별,37.0,볼(N),스치(P)
179253,발라드,2019.0,31453551.0,흔한 이별,37.0,바람(N),스치(P)
179254,발라드,2019.0,31453551.0,흔한 이별,37.0,바람(N),따르(P)


In [5]:
common_words.reset_index(drop=True).to_csv(
    "../data/lyrics_cooccured/lyric_cooccured_common_words_1995_2019.csv",
    encoding="utf-8-sig",
)


## 3. 5년 주기별 중복 상위 50% 노드 제거


In [6]:
duplicate_words = [
    key
    for key, _ in sorted(
        duplicate_nodes_cache.items(), key=lambda item: item[1], reverse=True
    )[: len(duplicate_nodes_cache) // 2]
]
top_half_duplicate_words = duplicate_words[: len(duplicate_words) // 2]
top_half_duplicate_words


[('말(N)', '하(P)'),
 ('사랑(N)', '하(P)'),
 ('사랑(N)', '없(P)'),
 ('곁(N)', '있(P)'),
 ('눈(N)', '감(P)'),
 ('손(N)', '잡(P)'),
 ('사랑(N)', '되(P)'),
 ('말(N)', '없(P)'),
 ('사랑(N)', '있(P)'),
 ('숨(N)', '쉬(P)'),
 ('눈물(N)', '흐르(P)'),
 ('나(N)', '나(P)'),
 ('가슴(N)', '아프(P)'),
 ('날(N)', '보(P)'),
 ('눈물(N)', '흘리(P)'),
 ('필요(N)', '없(P)'),
 ('일(N)', '없(P)'),
 ('이제(N)', '없(P)'),
 ('눈(N)', '뜨(P)'),
 ('사랑(N)', '알(P)'),
 ('말(N)', '있(P)'),
 ('맘(N)', '모르(P)'),
 ('바보(N)', '같(P)'),
 ('말(N)', '되(P)'),
 ('길(N)', '가(P)'),
 ('때(N)', '보(P)'),
 ('생각(N)', '나(P)'),
 ('눈물(N)', '나(P)'),
 ('사랑(N)', '모르(P)'),
 ('눈(N)', '오(P)'),
 ('꿈(N)', '꾸(P)'),
 ('눈(N)', '보(P)'),
 ('자신(N)', '없(P)'),
 ('생각(N)', '하(P)'),
 ('춤(N)', '추(P)'),
 ('곳(N)', '없(P)'),
 ('사람(N)', '만나(P)'),
 ('힘(N)', '들(P)'),
 ('시간(N)', '흐르(P)'),
 ('사랑(N)', '같(P)'),
 ('노래(N)', '부르(P)'),
 ('때(N)', '있(P)'),
 ('일(N)', '하(P)'),
 ('날(N)', '없(P)'),
 ('사람(N)', '좋(P)'),
 ('맘(N)', '알(P)'),
 ('말(N)', '듣(P)'),
 ('날(N)', '하(P)'),
 ('날(N)', '오(P)'),
 ('이상(N)', '없(P)'),
 ('사랑(N)', '주(P)'),


In [7]:
for word in word_fragments_by_year:
    word_fragments_no_duplicate = word.loc[
        list(
            map(
                lambda x: x not in top_half_duplicate_words,
                zip(word["noun"], word["predicate"]),
            )
        )
    ]
    file_name = f"lyric_cooccured_no_duplicate_words_{int(word['year'].min())}_{int(word['year'].max())}"
    print(file_name)
    word_fragments_no_duplicate.reset_index(drop=True).to_csv(
        f"../data/lyrics_cooccured/{file_name}.csv", encoding="utf-8-sig"
    )


lyric_cooccured_no_duplicate_words_1995_1999
lyric_cooccured_no_duplicate_words_2000_2004
lyric_cooccured_no_duplicate_words_2005_2009
lyric_cooccured_no_duplicate_words_2010_2014
lyric_cooccured_no_duplicate_words_2015_2019
